In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
tf.config.run_functions_eagerly(True)

from tensorflow.keras.utils import image_dataset_from_directory

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import regnet

# Initialize rng
rng = np.random.default_rng(2022)

auc = tf.keras.metrics.AUC()

In [3]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
batch_size = 32 # This is a tunable hyperparameter
shape = (128, 128) # note we are reducing the size of the image
# Note: you will use 'grayscale' images for your own model
# but you might need to switch to 'rgb' for pretrained models because they are trained on ImageNet which has only RGB images
data_dir = '/content/drive/MyDrive/Final_Project/Dataset'
train_ds_new = tf.keras.utils.image_dataset_from_directory(os.path.join(data_dir, 'train_new'),
                                                       seed=rng.integers(500000),
                                                       image_size=shape,
                                                       label_mode="categorical",
                                                       color_mode='grayscale',
                                                       batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(data_dir, 'validation/validation'),
                                                     seed=rng.integers(500000),
                                                     image_size=shape,
                                                     label_mode="categorical",
                                                     color_mode='grayscale',
                                                     batch_size=batch_size)
test_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(data_dir, 'test_new'),
                                                      seed=rng.integers(500000),
                                                      image_size=shape,
                                                      label_mode="categorical",
                                                      color_mode='grayscale',
                                                      batch_size=batch_size)

Found 16000 files belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Found 4000 files belonging to 4 classes.
Found 4000 files belonging to 4 classes.


In [5]:
for images, _ in train_ds_new.take(1):
    print("Image batch shape: ", images.shape)

Image batch shape:  (32, 128, 128, 1)


In [6]:
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds_new = train_ds_new.map(normalize)
val_ds = val_ds.map(normalize)
test_ds = test_ds.map(normalize)


In [7]:
from tensorflow.keras.layers.experimental import preprocessing

# Data augmentation layers
data_augmentation_layers = tf.keras.Sequential([
    preprocessing.RandomFlip("horizontal_and_vertical"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomZoom(0.2),
])

# Adding data augmentation to the training dataset
def augment_data(image, label):
    return data_augmentation_layers(image), label

train_ds_new = train_ds_new.map(augment_data)


In [1]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

def encoder(inputs):
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)  # Added dropout
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)  # Added dropout
    # Additional layer
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    return x

def decoder(encoded):
    x = Conv2DTranspose(128, (3, 3), strides=2, activation='relu', padding='same')(encoded)
    x = Dropout(0.25)(x)  # Added dropout
    x = Conv2DTranspose(64, (3, 3), strides=2, activation='relu', padding='same')(x)
    x = Dropout(0.25)(x)  # Added dropout
    x = Conv2DTranspose(32, (3, 3), strides=2, activation='relu', padding='same')(x)
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    return x


from tensorflow.keras.regularizers import l2

def classifier(encoded):
    x = Flatten()(encoded)
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)  # Added L2 regularization
    x = Dropout(0.5)(x)
    x = Dense(4, activation='softmax', kernel_regularizer=l2(0.001))(x)  # Added L2 regularization
    return x


input_img = Input(shape=(128, 128, 1))  # Make sure this shape matches your dataset

# Build the autoencoder
encoded = encoder(input_img)
decoded = decoder(encoded)

# Build the classifier
classification_output = classifier(encoded)

# Full model
autoencoder_classifier = Model(inputs=input_img, outputs=[decoded, classification_output])

# Compile the model with multiple outputs and loss functions




# Now you can proceed to train the model with train_ds_new and validate with val_ds
# Remember to use a one-hot encoded format for your labels if you're using 'categorical_crossentropy'


In [9]:
autoencoder_classifier.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 32)         320       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)           0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 dropout (Dropout)           (None, 64, 64, 32)           0         ['max_pooling2d[0][0]']   

In [10]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.round(y_pred)
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()


In [11]:
autoencoder_classifier.compile(
    optimizer='adam',
    loss=['mse', 'categorical_crossentropy'],  # mse for reconstruction, categorical_crossentropy for classification
    metrics={
        'conv2d_3': ['accuracy'],  # Assuming 'conv2d_11' is your decoder output - accuracy might not be meaningful here
        'dense_1': ['accuracy', Precision(), Recall(), F1Score()]  # Assuming 'dense_7' is your classification output
    }
)

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


In [12]:
train_ds_new = train_ds_new.map(augment_data).repeat()
val_ds = val_ds.repeat()

def data_generator(dataset):
    for images, labels in dataset:
        yield images, [images, labels]

train_generator = data_generator(train_ds_new)
val_generator = data_generator(val_ds)

train_batch_size = 32
val_batch_size = 32  # Can be different from train_batch_size if needed



In [13]:
train_steps_per_epoch = 16000 // batch_size
val_steps_per_epoch = 4000 // batch_size

# Adjust for any remaining images not fitting into a full batch
if 16000 % batch_size != 0:
    train_steps_per_epoch += 1
if 4000 % batch_size != 0:
    val_steps_per_epoch += 1




# Train the model using these generators
autoencoder_classifier.fit(
    train_generator,
    epochs=50,  # Or your desired number of epochs
    steps_per_epoch=train_steps_per_epoch,
    validation_data=val_generator,
    validation_steps=val_steps_per_epoch
)




Epoch 1/50
193/500 [==========>...................] - ETA: 10:16 - loss: 1.4924 - conv2d_3_loss: 0.0250 - dense_1_loss: 1.3893 - conv2d_3_accuracy: 0.0078 - dense_1_accuracy: 0.2581 - dense_1_precision: 0.2222 - dense_1_recall: 6.4767e-04 - dense_1_f1_score: 0.0013

KeyboardInterrupt: ignored

In [126]:
# Adjust the test dataset
def prepare_test_data(images, labels):
    # Ensure the structure is correct: (images, [images, labels])
    return images, (images, labels)

# Apply the function to the test dataset
test_data = test_ds.map(prepare_test_data)


# Evaluate the model
results = autoencoder_classifier.evaluate(test_data)
print(f"Test Loss: {results[0]}")
print(f"Test Reconstruction Loss: {results[1]}")
print(f"Test Classification Loss: {results[2]}")
print(f"Test Reconstruction Accuracy: {results[3]}")
print(f"Test Classification Accuracy: {results[4]}")
print(f"Test Precision: {results[5]}")
print(f"Test Recall: {results[6]}")
print(f"Test F1 Score: {results[7]}")  # F1 score result



125/125 [==============================] - 9s 72ms/step - loss: 1.0356 - conv2d_46_loss: 0.0106 - dense_29_loss: 0.9673 - conv2d_46_accuracy: 0.0367 - dense_29_accuracy: 0.5698 - dense_29_precision_16058: 0.7314 - dense_29_recall_16058: 0.3077 - dense_29_f1_score: 0.4332
Test Loss: 1.0356038808822632
Test Reconstruction Loss: 0.010560151189565659
Test Classification Loss: 0.967251181602478
Test Reconstruction Accuracy: 0.03667861968278885
Test Classification Accuracy: 0.5697500109672546
Test Precision: 0.7314319610595703
Test Recall: 0.30774998664855957
Test F1 Score: 0.43322181701660156
